In [ ]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

In [ ]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [ ]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [ ]:
# View all of the classes that automap found
Base.classes.keys()

In [ ]:
# Save references to each table
measurementsTable = Base.classes.measurement
stationTable = Base.classes.station

In [ ]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Exploratory Precipitation Analysis

In [ ]:
# Find the most recent date in the data set.
#most_recent = session.query("select top 1 * from station order by date desc ")
most_recent = session.query(func.max(measurementsTable.date)).first()
print(f'The most recent date was {most_recent}')

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
# Starting from the most recent data point in the database. 
# Calculate the date one year from the last date in data set.
latest_date = dt.date(2017, 8, 23)
last_year = latest_date - dt.timedelta(days = 365)
lastTwelveMonths = session.query(measurementsTable).\
    filter(measurementsTable.date > last_year)
print(lastTwelveMonths.count())


# Perform a query to retrieve the data and precipitation scores
precipitation_Array = []
date_Array = []
for measurement in lastTwelveMonths:
    precipitation_Array.append(measurement.prcp)
    date_Array.append(measurement.date)
# Save the query results as a Pandas DataFrame and set the index to the date column
precipFrame = pd.DataFrame({'Date': date_Array, 'Precipitation': precipitation_Array})
# Sort the dataframe by date
precipFrame.sort_values(by=['Date'], ascending=False)

# Use Pandas Plotting with Matplotlib to plot the data
precipFrame.plot(x='Date',y='Precipitation', rot=90)
plt.xlabel("Date")
plt.ylabel("Precipitation, in Metric Inches")
plt.show()
#I prefer Scatter

In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data
precipFrame.describe()
#here I was about to write out an analysis in human language.

# Exploratory Station Analysis

In [ ]:
# Design a query to calculate the total number stations in the dataset
# Using data from Automap step previously
stations_total = session.query(measurementsTable).group_by(measurementsTable.station).count()
print(f"Total number of stations: {stations_total}")

In [ ]:
# Design a query to find the most active stations (i.e. what stations have the most rows?)
# List the stations and the counts in descending order.
active_stations = session.query(measurementsTable.station, func.count(measurementsTable.id))
most_activestations = active_stations.group_by(measurementsTable.station).order_by(func.count(measurementsTable.id).desc()).all()
most_activestations

In [ ]:
# Using the most active station id from the previous query, calculate the lowest, highest, and average temperature.
mostActive = session.query(measurementsTable.station).group_by(measurementsTable.station).order_by(func.count(measurementsTable.id).desc()).first()
stationId = mostActive[0]
lowest = (session.query(func.min(measurementsTable.tobs)).filter(measurementsTable.station == stationId).all())[0][0]
highest = (session.query(func.max(measurementsTable.tobs)).filter(measurementsTable.station == stationId).all())[0][0]
ave_temp = (session.query(func.avg(measurementsTable.tobs)).filter(measurementsTable.station == stationId).all())[0][0]
print(f"Most active station is {stationId} with a low of {lowest}, a high of {highest}, and an average across all recorded measurements of {ave_temp}")

In [ ]:
# Using the most active station id
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
topMeasurements = session.query(measurementsTable.tobs, measurementsTable.station).filter(measurementsTable.date >= last_year).filter(measurementsTable.station == stationId).all()

# Save the query results as a Pandas DataFrame and set the index to the station column
topStationdf = pd.DataFrame(topMeasurements)
topStationdf.set_index('station', inplace=True)


#Plot results as a histogram
topStationdf.hist(bins = 16)
plt.xlabel("Temperature")
plt.ylabel("Frequency")
plt.title("Frequency of Temperatures in Most Active Station")
plt.tight_layout()
plt.show()


# Close session

In [ ]:
# Close Session
session.close()
#Goodnight!